# Modelling - Name Entity Recognition
## Portal da Queixa
### Changes may be required due to Portal da Queixa's continous updates

Lefties Group

In [1]:
# Import packages
import csv
import pandas as pd
import numpy as np
import nltk 
import re
from bs4 import BeautifulSoup
import spacy
from spacy import displacy 
from collections import Counter 


In [2]:
# Load dataset and assign the datatypes
dtypes = {'controlID':'category','complainID':'category','complainStatus':'category','complainUser':'category',
  'complainTitle':'category','complainViews':'category','complainText':'category',
  'complainDate':'category'}
df = pd.DataFrame(pd.read_excel("complains_data.xlsx", dtype=dtypes, index_col='complainID'))

In [3]:
# Text preprocessing
def textPreProcess(rawText, removeHTML=True, charsToRemove = r'\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|\:|\-\_x000d_\n', removeNumbers=True, removeLineBreaks=False, specialCharsToRemove = r'[^\x00-\xfd]', convertToLower=True, removeConsecutiveSpaces=True):
    cleanedText = []
    for x in (rawText[:]): 
        
        # Remove HTML
        if removeHTML:
            procText = BeautifulSoup(x,'html.parser').get_text()

         # Remove punctuation and other special characters
        if len(charsToRemove)>0:
            procText = re.sub(charsToRemove,' ',procText)

        # Remove numbers
        if removeNumbers:
            procText = re.sub(r'\d+',' ',procText)

        # Remove line breaks
        if removeLineBreaks:
            procText = procText.replace('\n',' ').replace('\r', '')

        # Remove special characters
        if len(specialCharsToRemove)>0:
            procText = re.sub(specialCharsToRemove,' ',procText)

        # Normalize to lower case
        if convertToLower:
            procText = procText.lower() 

        # Replace multiple consecutive spaces with just one space
        if removeConsecutiveSpaces:
            procText = re.sub(' +', ' ', procText)

        # If there is a text, add it to the clean text         
        if procText != '':
            cleanedText.append(procText)
    return cleanedText

In [4]:
# Create a dataframe with only the description, not removing because upper case can identify important things
ppText = textPreProcess(df.complainText, charsToRemove='', removeLineBreaks=False, removeNumbers=False)
processedComplains =  pd.DataFrame(data=ppText, index= df.index, columns=['PreProcessedText']) 


In [5]:
# Load Spacy Portuguese model
nlp = spacy.load("pt_core_news_sm")

In [6]:
# Check entities in complain 
print(processedComplains['PreProcessedText'][27355319])
doc = nlp(processedComplains['PreProcessedText'][27355319])
print([(X.text, X.label_) for X in doc.ents])

no dia 11 de abril de 2018, fui a lefties do strada odivelas parque, desde que tinha entrada na loja deparei-me com uma funcionaria da loja que andava a tras da minha pessoa por onde eu iria com desconfiança que eu iria roubar. a principio não levei em conta, mas depois de alguns minutos reparei que a mesma funcionaria dava sinal as colegas para que chamasse o segurança. depois de alguns instante reparei que havia mais duas meninas de raça negra na loja que também a mesma funcionaria tinha dado sinal as colegas para que ficassem de vigia a tras das mesma. senti-me desconfortável e chamei a funcionária para lhe dar uma palavra, a mesma dirigiu-se a mim dizendo que estava a trabalhar que estava ocupada que não me podia atender. eu dirigi-me a funcionaria e disse-lhe que precisava de um esclarecimento e a funcionaria foi de todo mal intencionada demostrando visivelmente o seu acto discriminatorio perante a mim. no final disso tudo pedi o livro de reclamação e a identificação a mesma negou

In [7]:

#Create counter and and empty list for the complains mentioning locations in a for loop
counter=0  
locComplains=[]

for r in processedComplains['PreProcessedText']:
  doc = nlp(r)
  for i in doc.ents:
      if i.label_=='LOC':
          locComplains.append(r)
          counter = counter + 1
          break
  if counter>=5:    # Stop after the first five complains
      break

locComplains #Print the 5 complains involving a location

['boa noite, _x000d_\nvenho mostrar o meu total descontentamento e frustação relativamente ao serviço prestado. _x000d_\nfiz uma encomenda de várias peças, uma delas um pack de 4 calças de fato de treino por 15,99 . ao chegar a encomenda deparei-me que em vez do pack só tinha recebido 1 calça em vez das 4. enviei e-mail para o apoio ao cliente a expor a situação, passado algumas horas responderam a afirmar que teriam entrado em contacto comigo várias vezes sem sucesso (o que é mentira pois confirmei o número de telefone cedido na minha conta e não tinha nenhuma chamada não atendida), pediram para retribuir a chamada. _x000d_\ncontactei então por chamada telefonica, ao qual a assistente que me atendeu nunca pediu desculpa pelo sucedido mas explicou que deveria devolver a calça com uma etiqueta própria e que me seria enviado o pack completo. _x000d_\npassado dois dias recebi o pack mas desta vez 1 das 4 calças veio com vários tamanhos abaixo do pedido. _x000d_\nquero que resolvam esta si

In [8]:
#Create counter and and empty list for the complains mentioning organizations in a for loop
counter=0  
orgComplains=[]

for r in processedComplains['PreProcessedText']:
  doc = nlp(r)
  for i in doc.ents:
      if i.label_=='ORG':
          orgComplains.append(r)
          counter = counter + 1
          break
  if counter>=5:    # Stop after the first five complains, but could be more
      break

orgComplains #Print the complains found

['a minha encomenda que fiz no dia 19 de março não foi entregue devido os ctt expresso não me ter entregue a casa.disseram que houve tentativa de entrega quando eu estava em casa e não recebi nenhuma forma de comunicação da parte deles.liguei para o apoio ao cliente e este disseram me que tinha a transportadora e deveria voltar para trás, a lefties.eu garanti que assim fosse mas disse que queria o reembolso da mesma.disseram me que tinham que confirmar com a transportadora.eu esperei jma semana e voltei a ligar,estes disseram me que iriam devolver o valor no paypal mas continuei à espera e nada.voltei a ligar e o operador dizia me que me iam reembolsar mas continuava à espera e nada. até que voltei a ligar e uma senhora disse me que já estava à muito tempo à espera desde dia 09 que foi devolvido e pediu para aguardar.voltei a ligar no dia 13 e a partir desse dia disseram me que levaria 14 dias a contar a partir do dia 09.voltei a ligar no dia 19,e uma senhora disse me que iam me enviar